<a href="https://colab.research.google.com/github/Himanshukunar/excelr-assignments/blob/main/Assignment18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
df = pd.read_csv("Alphabets_data.csv")  # Use correct file name

# Basic summary
print("Data Shape:", df.shape)
print("Column Names:", df.columns)
print(df.head())

# Handle missing values
df.dropna(inplace=True)

# Separate features and target
X = df.drop('letter', axis=1)
y = df['letter']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

Data Shape: (20000, 17)
Column Names: Index(['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar',
       'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey',
       'yedge', 'yedgex'],
      dtype='object')
  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build basic ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(26, activation='softmax')  # 26 classes for A-Z
])

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1, verbose=1)

# Evaluate
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test, y_pred_labels, target_names=label_encoder.classes_))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2816 - loss: 2.6381 - val_accuracy: 0.6700 - val_loss: 1.3405
Epoch 2/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6940 - loss: 1.2091 - val_accuracy: 0.7462 - val_loss: 0.9603
Epoch 3/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7543 - loss: 0.9246 - val_accuracy: 0.7869 - val_loss: 0.8074
Epoch 4/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7905 - loss: 0.7748 - val_accuracy: 0.8106 - val_loss: 0.7068
Epoch 5/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8175 - loss: 0.6713 - val_accuracy: 0.8131 - val_loss: 0.6431
Epoch 6/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8250 - loss: 0.6265 - val_accuracy: 0.8300 - val_loss: 0.5884
Epoch 7/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8418 - loss: 0.5636 - val_accuracy: 0.8475 - val_loss: 0.5468
Epoch 8/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8502 - loss: 0.5146 - val_accuracy: 0.

In [4]:
!pip install -q keras-tuner

from kerastuner.tuners import RandomSearch
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units1', min_value=32, max_value=128, step=32),
                    activation=hp.Choice('act1', ['relu', 'tanh']),
                    input_shape=(X_train.shape[1],)))
    model.add(Dense(units=hp.Int('units2', 32, 128, step=32),
                    activation=hp.Choice('act2', ['relu', 'tanh'])))
    model.add(Dense(26, activation='softmax'))

    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='ann_tuning',
    project_name='alphabets'
)

tuner.search(X_train, y_train, epochs=20, validation_split=0.1, verbose=1)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate tuned model
y_pred_tuned = best_model.predict(X_test)
y_pred_labels_tuned = np.argmax(y_pred_tuned, axis=1)

print("\nTuned Model Accuracy:", accuracy_score(y_test, y_pred_labels_tuned))
print("\nTuned Classification Report:\n", classification_report(y_test, y_pred_labels_tuned, target_names=label_encoder.classes_))


Trial 10 Complete [00h 00m 32s]
val_accuracy: 0.9506250023841858

Best val_accuracy So Far: 0.9537500143051147
Total elapsed time: 00h 05m 06s
  1/125 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Tuned Model Accuracy: 0.94775

Tuned Classification Report:
               precision    recall  f1-score   support

           A       0.98      0.99      0.98       149
           B       0.90      0.92      0.91       153
           C       0.98      0.94      0.96       137
           D       0.94      0.96      0.95       156
           E       0.91      0.94      0.93       141
           F       0.93      0.92      0.93       140
           G       0.93      0.95      0.94       160
           H       0.87      0.85      0.86       144
           I       0.90      0.97      0.94       146
           J       1.00      0.93      0.97       149
           K       0.86      0.95      0.90       130
           L       0.98      0.94      0.96       155
           M       0.97      0.97      0.97       168
           N       0.98      0.93      0.95       151
           O       0.93      0.97      0.95       145
           P       0.99      0.9